In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, convert_to_messages
from os.path import dirname, join
from dotenv import load_dotenv
from typing import Literal
from typing_extensions import TypedDict, Annotated
from langgraph.graph import StateGraph, START, MessagesState, END
from langgraph.types import Command
from operator import add
import pandas as pd
import getpass
import os

In [ ]:
# Set your OpenAI API key
load_dotenv("../.env")

if not os.environ.get("OPENAI_API_KEY"): #field to ask for OpenAI API key
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter OpenAI API Key: ")

In [ ]:
from Agent import BTEx

In [ ]:
# Create an LLM-based agent
llm = ChatOpenAI(temperature=0, model="gpt-4o")  # Change model if needed

In [ ]:
def get_llm_response(question):
    return llm.invoke(question).content

In [ ]:
drugDiseaseDMDB = pd.read_csv('./data/DMDB_go_bp_filtered_jjoy_05_08_2025.csv')

In [ ]:
drugDiseaseSet = drugDiseaseDMDB.sample(50)

print(drugDiseaseSet)

In [ ]:
# Not including identifiers in the question

drugDiseaseSet['question w/o ID'] = "Which drugs can treat " + drugDiseaseSet['disease_name'] + " by targeting " + drugDiseaseSet['bp_name'] + "? Return a comprehensive list of all drugs that target the correct biological process"

In [ ]:
# Including identifiers in question

drugDiseaseSet['question w/ ID'] = "Which drugs can treat " + drugDiseaseSet['disease_name'] + " (ID: " + drugDiseaseSet['disease'] + ") by targeting " + drugDiseaseSet['bp_name'] + " (ID: " + drugDiseaseSet['bp'] + ")? Return a comprehensive list of all drugs that target the correct biological process"

In [ ]:
# LLM answer w/o identifiers

drugDiseaseSet['gpt4.0_response w/o ID'] = drugDiseaseSet["question w/o ID"].apply(get_llm_response)

In [ ]:
# LLM answer w/ identifiers

drugDiseaseSet['gpt4.0_response w/ ID'] = drugDiseaseSet["question w/ ID"].apply(get_llm_response)

In [ ]:

drugDiseaseSet.to_excel('./logs/drug <- diseasebp/save1_50questions_drugfromdiseasebp_08.05.25.xlsx')

In [ ]:
# BTE-RAG w/o ID (maxresults: 100, k = 5)

drugDiseaseSet["BTEx_response w/o ID (maxresult = 100, k = 5)"] = drugDiseaseSet["question w/o ID"].apply(lambda q: BTEx(q, maxresults=100, k=5))

In [ ]:
# BTE-RAG w/ ID (maxresults: 100, k = 5)

drugDiseaseSet["BTEx_response w/ ID (maxresult = 100, k = 5)"] = drugDiseaseSet["question w/ ID"].apply(lambda q: BTEx(q, maxresults=100, k=5))

In [ ]:
# BTE-RAG w/o ID (maxresults: 100, k = 10)

drugDiseaseSet["BTEx_response w/o ID (maxresult = 100, k = 10)"] = drugDiseaseSet["question w/o ID"].apply(lambda q: BTEx(q, maxresults=100, k=10))

In [ ]:
# BTE-RAG w/ ID (maxresults: 100, k = 10)

drugDiseaseSet["BTEx_response w/ ID (maxresult = 100, k = 10)"] = drugDiseaseSet["question w/ ID"].apply(lambda q: BTEx(q, maxresults=100, k=10))

In [ ]:
# BTE-RAG w/o ID (maxresults: 100, k = 15)

drugDiseaseSet["BTEx_response w/o ID (maxresult = 100, k = 15)"] = drugDiseaseSet["question w/o ID"].apply(lambda q: BTEx(q, maxresults=100, k=15))

In [ ]:
# BTE-RAG w/ ID (maxresults: 100, k = 15)

drugDiseaseSet["BTEx_response w/ ID (maxresult = 100, k = 15)"] = drugDiseaseSet["question w/ ID"].apply(lambda q: BTEx(q, maxresults=100, k=15))

In [ ]:
# Save responses

drugDiseaseSet.to_excel('./logs/drug <- diseasebp/save2_50questions_drugfromdiseasebp_08.05.25.xlsx')